# TWW 

**Tanzania Water Wells** quality classification.

In [ ]:
# Basic Libs
import pandas as pd
import numpy as np

# DataViz Libs
import matplotlib.pyplot as plt
import seaborn as sns

# Modeling
from sklearn.impute import KNNImputer
from sklearn.model_selection import train_test_split
from catboost import CatBoostClassifier, Pool

# Notebook Setup
from warnings import simplefilter
simplefilter('ignore')
pd.set_option('display.max_columns', None)

## Loading Datasets

In [ ]:
# Load Datasets
X = pd.read_csv('../data/X_train.csv')
y = pd.read_csv('../data/y_train.csv')

In [ ]:
# Meet the dataset
X.head(3)

### Checks

**Train and test datasets have been provided separately.**

Let's see if they satisfy some conditions.

The conditions are:

- **Both datasets have the same number of rows;**
- **Both datasets are _sorted_ equally in order or appearance;**

If these conditions are not met, we will need some basic data wrangling before proceeding.

#### Number of Rows

In [ ]:
# Check if the number of rows in both X and y are the same
X.shape, y.shape

Both datasets have the same number of rows, which is a good sign.

Now we can investigate the second condition, about their order of appearance.

#### Equally sorted

In [ ]:
(X['id'].values == y['id'].values).all()

In [ ]:
print(y['id'].values[:5])
print(X['id'].values[:5])

`X` and `y` are _sorted_ equally. The second condition has also been met.

The integrity of the datasets have been confirmed.

At the moment of training a classification model for the task we will use them separately, but there are still some **Exploratory Data Analysis (EDA)** ahead. If we keep them separate and drop some rows in `X` dataset, for example, we will need to perform the same operation for the corresponding indexes in `y`. 

It's more convenient to merge both datasets into a simple `df` python variable.

### Merge `X` and `y` into `df`

In [ ]:
# Merge X and y
df = X.merge(y, on = 'id')
df.head(3)

## EDA

**Exploratory Data Analysis is an extended task for Modeling Data**. 

Let's do some EDA on data integrity rather than statistical significance for now.

### Shape

In [ ]:
# Check df shape
df.shape

### Metadata

In [ ]:
# Check df metadata
df.info()

One of the easiest things to notice in this dataset is that:

1. `date_recorded` variable is stored as a `pandas` `object` instead of `datetime` object. We will handle this shortly;
2. Some variables have `null` values;

Other data types issues, harder to spot, may refer to `bool` variables recorded as object. We will investigate on that either.

But first, it's convenient to keep track of what needs to be done in a TO-DO list. Let's build it.

</br><center>------</center></br>

As of now, our TO-DO list looks like this:

**TO-DO**
- [ ] parse `date_recorded` to `pd.datetime` data type;
- [ ] investigate on possible wrongly parsed data types for `object` variables;
- [ ] deal with `null` values.


#### Handling `date_recorded` variable data type

In [ ]:
# Change date_recorded to datetime dtype
df['date_recorded'] = pd.to_datetime(df['date_recorded'])

#### Investigate other data types

In [ ]:
categorical_vars = X.select_dtypes(include = 'object', exclude = 'datetime64').columns.tolist()

for var in categorical_vars:
    if len(X[var].value_counts()) <= 2:
        print(var.center(34, '-'))
        print(X[var].value_counts(dropna = False), '\n')
        print(f'Percent (%) of NaNs: {(X[var].isna().mean())*100:.2f}%'.center(34))
        print("-".center(34, '-'), '\n')
        

`public_meeting` and `permit` would have been parsed as `bool` if it wasn't for the `nan` values. We will deal with that when dealing with NAs values shortly.

`recorded_by` has a single value across all dataset. This variable is redundant and would add extra complexity to the model that would not result in a model's performance improvement. Let's drop it.

Before. Let's update our TO-DO list.

</br><center>------</center></br>

Updated TO-DO list:

**TO-DO**
- [x] [parse `date_recorded` to `pd.datetime` data type;](#Handling-date_recorded-variable-data-type)
- [ ] investigate on possible wrongly parsed data types for `object` variables;
    - [x] [review `df` and eyespot wrongly parsed datatypes;](#Loading-Datasets)
    - [ ] drop `recorded_by` due to redundancy;
    - [ ] check `category` variables unique number of classes;
- [ ] deal with `null` values.


### Drop `recorded_by`

In [ ]:
df = df.drop(columns=['recorded_by'])

Updated TO-DO list:

**TO-DO**
- [x] [parse `date_recorded` to `pd.datetime` data type;](#Handling-date_recorded-variable-data-type)
- [ ] investigate on possible wrongly parsed data types for `object` variables;
    - [x] [review `df` and eyespot wrongly parsed datatypes;](#Loading-Datasets)
    - [x] [drop `recorded_by` due to redundancy;](#Drop-recorded_by)
    - [ ] check `category` variables unique number of classes;
- [ ] deal with `null` values.


### Check `category` variables

In [ ]:
categorical_vars = df.select_dtypes(include=[object, 'category']).columns.tolist()

plt.figure(figsize=(15, 3))
df[categorical_vars].nunique().hist(color = 'gray', bins = 100)
sns.despine(left = True)
plt.title('Histogram Count of Unique Classes for Categorical Vars')
plt.grid()
plt.tight_layout()

Most variables have few unique classes. But some of them have more than 2000 classes. Some variable have around 20,000 unique classes or more.

Modeling with that amount of classes will be difficult. Let's investigate on those variables a little closer.

In [ ]:
df[categorical_vars].nunique()[(df[categorical_vars].nunique() > 30)]

In [ ]:
y['status_group'].value_counts()

### NAs

In [ ]:
# Check null values
plt.figure(figsize = (15, 6))
df.isna().mean().plot(kind='bar', color = 'darkgray', zorder = 2)
sns.despine(left = True)
plt.grid(which = 'major', axis = 'y', color='lightgray', linestyle='--', linewidth=.5, zorder = 1)
plt.tight_layout()

Both **`public_meeting`** and **`permit`** variables appear as `categorical` data type, but they actually are `bool`. In fact, if it wasn't for the `'NaN'` values, they would have been parsed correctly yet while reading the file.

We need to either drop those lines or impute a thoughtful value. We'll consider this task ahead, but first let's take note of our tasks on helpful a **TO-DO list**.


</br><center>------</center></br>

**TO-DO**
- [x] parse `date_recorded` to `pd.datetime` data type
- [ ] drop or impute values to `public_meeting` and `permit`variables;
- [ ] drop `schema_name` variable;
- [ ] remove `recorded_by` columns due to that all values are the same `GeoDataConsultants Ltd`.


In [ ]:
df = df.dropna(subset = ['public_meeting', 'permit'])
df.shape

In [ ]:
plt.figure(figsize = (15, 6))
df.isna().mean().plot(kind='bar', color = 'darkgray', zorder = 2)
sns.despine(left = True)
plt.grid(which = 'major', axis = 'y', color='lightgray', linestyle='--', linewidth=.5, zorder = 1)
plt.tight_layout()

In [ ]:
df = df.drop(columns = 'scheme_name')
df.shape

In [ ]:
# Check y values
y['status_group'].value_counts(normalize=True)

### Modeling

Add to import cell:

```python
from sklearn.model_selection import train_test_split
from catboost import CatBoostClassifier, Pool
```

In [ ]:
# def train_test_eval_split(X, y, random_state = None, 
#                           train_size = .6, 
#                           test_size = .2, 
#                           val_size = .2,
#                           **kwargs):
    
#     df_n_rows = X.shape[0]
    
#     train_n_rows = np.ceil(df_n_rows * train_size)
#     val_n_rows = np.ceil(df_n_rows * val_size)
#     test_n_rows = np.floor(df_n_rows * test_size)
    
#     assert train_n_rows + val_n_rows + test_n_rows == df_n_rows
    
#     X_train, X_val_test, y_train, y_val_test = train_test_split(X, y, train_size = train_n_rows)
#     X_val, X_test, y_val, y_test = train_test_split(X_val_test, y_val_test, train_size = val_n_rows)
    
#     return X_train, X_val, X_test, y_train, y_val, y_test

In [ ]:
df = df.dropna(axis = 0)

In [ ]:
X = df.drop(columns = ['id', 'status_group'])
y = df['status_group']

In [ ]:
# train_test_eval_split(X, y)

In [ ]:
X_train, X_val_test, y_train, y_val_test = train_test_split(X, y, train_size = .6, random_state = 0)

In [ ]:
X_val, X_test, y_val, y_test = train_test_split(X_val_test, y_val_test, test_size = .5, random_state = 0)

In [ ]:
print(X_train.shape, X_val.shape, X_test.shape)
print(y_train.shape, y_val.shape, y_test.shape)

In [ ]:
train_dataset = Pool(X_train, y_train, cat_features = X_train.select_dtypes(include = [object, 'category']).columns.tolist())

In [ ]:
model_params = {
    'iterations': 1_000, 
    'loss_function': 'MultiClass', 
    'train_dir': 'crossentropy',
    'allow_writing_files': False,
    'random_seed': 42,
}

model = CatBoostClassifier(**model_params)
model.fit(train_dataset, verbose=True, plot=True)

In [ ]:
y_pred_val = model.predict(X_val)

In [ ]:
from sklearn.metrics import plot_confusion_matrix, classification_report

In [ ]:
print(plot_confusion_matrix(model, X_val, y_val))

In [ ]:
print(classification_report(y_val, y_pred_val))

### Impute missing values

```python 
### from sklearn.impute import KNNimmputer
```

Even if the amount of total rows missing is at most 3334 observations (which represents 5.6% of our dataset), I would like to try a `KNNImputer` on them.

In [ ]:
knn_imputer = KNNImputer()

In [ ]:
X[['public_meeting', 'permit']].values.astype(bool)

In [ ]:
X_knnimputer_train = X.sample(300)
index_X_knnimputer_train = X_knnimputer_train.index

y_knnimputer_train = X_knnimputer_train[['public_meeting', 'permit']]
y_permit_knnimputer_train = X_knnimputer_train['permit']

X_knnimputer_train = X_knnimputer_train.drop(columns = ['public_meeting', 'permit'])

In [ ]:
knn_imputer.fit(X_knnimputer_train, y=y_knnimputer_train)

In [ ]:
(3334 / X.shape[0]) * 100

In [ ]:
import xgboost as xgb

In [ ]:
xgb.XGBRFClassifier(use_label_encoder=False)

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
RandomForestClassifier(n_estimators=10_000)